In [1]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from scipy.ndimage import rotate
%matplotlib qt
from tqdm import tqdm
import psf
from matplotlib import colormaps as cmaps
from scipy.signal import convolve2d

In [2]:
img = Image.open("BeadStack_20170403_S1F1_01.tif")

In [3]:
# plt.close("all")
arr=[]
for i in range(600,1001):
    img.seek(i)
    arr.append(np.array(img))

In [4]:
for i, arr in enumerate(arr):
    plt.clf()
    plt.title(i)
    plt.imshow(arr,cmap="gray",vmin=20000,vmax=23000)
    plt.colorbar()
    plt.pause(1/25)

In [5]:
arr[0]

array([20187, 19802, 19689, 20133, 20328, 19661, 19792, 19644, 19415,
       19869, 20010, 19874, 19844, 19676, 20379, 19826, 20117, 19861,
       20162, 19845, 20020, 19929, 20073, 19627, 20167, 20242, 19729,
       19765, 19789, 19857, 20361, 20327, 19694, 20226, 19708, 19427,
       19507, 19906, 20492, 19817, 19690, 19922, 19971, 19939, 19723,
       19916, 19517, 20081, 19709, 20046, 19765, 19541, 20090, 20181,
       19600, 20134, 20041, 19846, 19508, 20203, 19970, 19950, 20010,
       20276, 20089, 20129, 19975, 19700, 19898, 19722, 19608, 19999,
       19733, 20029, 20041, 19906, 20094, 19743, 19737, 20126, 19865,
       20292, 20279, 20203, 20245, 20345, 20098, 19900, 20091, 19729,
       20001, 19741, 19781, 19834, 19779, 20035, 19827, 20078, 19929,
       20023, 20045, 20014, 20098, 20057, 20256, 20215, 19648, 20389,
       19731, 19852, 20410, 20000, 20143, 20098, 19651, 20172, 20078,
       19895, 19823, 20045, 20188, 19868, 19695, 19734, 20379, 20101,
       20187, 19580,

In [6]:
mask = np.zeros_like(arr[0],dtype=bool)
mask[len(mask)//2-2:len(mask)//2+1] = np.ones_like(mask[0],dtype=bool)
cross_sections_arr = []
for a in tqdm(arr):
    cross_sections = [rotate(a, deg,reshape=False)[mask].reshape(3,-1).mean(axis=0) for deg in np.linspace(0,360,6)]
    cross_sections = np.array(cross_sections)
    cross_sections_arr.append(np.mean(cross_sections,axis=0))
cross_sections_arr = np.array(cross_sections_arr)



  0%|          | 0/512 [00:00<?, ?it/s]


ValueError: input array should be at least 2D

In [7]:
plt.figure()
# plt.axis("off")
plt.imshow(cross_sections_arr,cmap=cmaps.get_cmap('cubehelix'),aspect=0.5,vmin=1.7e4)
plt.colorbar()
# plt.savefig("test_stack.png")
plt.show()

TypeError: Invalid shape (0,) for image data

In [8]:

args = dict(
     shape=(51, 51),
     dims=(2, 5),
     ex_wavelen=488,
     em_wavelen=520,
     num_aperture=1.2,
     refr_index=1.333,
     pinhole_radius=0.5,
     pinhole_shape='round',
 )
# obsvol = psf.PSF(**args)
# obsvol.sigma.ou
obsvol = psf.PSF( psf.ISOTROPIC|psf.EMISSION, **args)
# plt.close(fig)
psfunc = (psf.mirror_symmetry(obsvol.data))
x0,y0=len(psfunc)/2,len(psfunc)/2
def potential(x,y,r):
    r_sq = ((x-x0)**2+(y-y0)**2)
    cond = r_sq > r**2 and r_sq <= (r+1)**2
    if cond:
        print(r_sq,r**2)
    return 1 if cond else 0
obj = np.zeros_like(psfunc)
for (i,j),val in np.ndenumerate(obj):
    obj[i,j] = potential(i,j,30)
plt.imshow(obj)


960.5 900
950.5 900
942.5 900
936.5 900
932.5 900
930.5 900
930.5 900
932.5 900
936.5 900
942.5 900
950.5 900
960.5 900
960.5 900
942.5 900
926.5 900
912.5 900
900.5 900
900.5 900
912.5 900
926.5 900
942.5 900
960.5 900
944.5 900
922.5 900
902.5 900
902.5 900
922.5 900
944.5 900
938.5 900
912.5 900
912.5 900
938.5 900
942.5 900
912.5 900
912.5 900
942.5 900
956.5 900
922.5 900
922.5 900
956.5 900
942.5 900
906.5 900
906.5 900
942.5 900
932.5 900
932.5 900
926.5 900
926.5 900
924.5 900
924.5 900
926.5 900
926.5 900
932.5 900
932.5 900
942.5 900
942.5 900
956.5 900
906.5 900
906.5 900
956.5 900
922.5 900
922.5 900
942.5 900
942.5 900
912.5 900
912.5 900
938.5 900
938.5 900
912.5 900
912.5 900
944.5 900
944.5 900
922.5 900
922.5 900
960.5 900
902.5 900
902.5 900
960.5 900
942.5 900
942.5 900
926.5 900
926.5 900
912.5 900
912.5 900
960.5 900
900.5 900
900.5 900
960.5 900
950.5 900
950.5 900
942.5 900
942.5 900
936.5 900
936.5 900
932.5 900
932.5 900
930.5 900
930.5 900
930.5 900
930.5 900


In [ ]:
fig =  plt.figure()
# plt.clf()
plt.imshow((convolve2d(obj,np.log(psfunc),mode="same")))
plt.show()

/var/folders/39/51hl1qv962dgjs7r9mznnsxm0000gp/T/ipykernel_66614/2542102357.py:3: RuntimeWarning: invalid value encountered in log
  plt.imshow((convolve2d(obj,np.log(psfunc),mode="same")))


In [ ]:
np.min(psfunc)

1.4692038540816528e-05